# CBF Tutorial - Part 2: Dynamic Obstacles

 Authors: Bardh, Tom

## Introduction

In part 2 of this tutorial series, we show how we can use CBFs to navigate a system around dynamic obstacles. We assume that the dynamics of the obstacles are known.

## Tutorial

Again we start with the **nimble_ant** system, which has the following dynamics:
$$
  \dot{xr} = u = \begin{bmatrix}u_0 \\ u_1\end{bmatrix}
$$
Given an initial position $x^{init}$, our goal is to control this system to a goal location $xr^{goal}$.
$$
xr^{init} = \begin{bmatrix} xr^{init}_0 \\  xr^{init}_1 \end{bmatrix} \qquad	xr^{goal} = \begin{bmatrix} xr^{goal}_0 \\  xr^{goal}_1 \end{bmatrix}
$$
However, different from our last example, the obstacles are dynamic. That means that our CBF depends not only on the dynamics of the system, but the obstacles as well. Recall our definition of the Barrier Function:
$$
\dot{B}(x) \ge -\alpha\cdot(B(x))
$$
In this case, the barrier function is defined as follows:

$$
B(xr,xo) = \frac{(xr_0-xo_0)^2}{a^2} + \frac{(xr_1-xo_1)^2}{b^2} - 1 \tag{1}
$$
where the robot states $xr$ and the obstacle states $xo$.
To obtain the derivative, we utilize [multivariate chain rule](https://en.wikipedia.org/wiki/Chain_rule#Multivariable_case):

$$
\dot{B}(xr,xo) = \frac{\partial{B}}{\partial{xr}} \cdot \frac{\partial{xr}}{\partial{t}} + \frac{\partial{B}}{\partial{xo}} \cdot \frac{\partial{xo}}{\partial{t}}
$$

Since $\frac{\partial{xr}}{\partial{t}} = \dot{xr}$, $\frac{\partial{xo}}{\partial{t}} = \dot{xo}$ then

$$
\dot{B}(xr,xo) = \frac{\partial B}{\partial xr} \cdot \dot{xr} + \frac{\partial B}{\partial xo} \cdot \dot{xo} \tag{2}
$$
this is equivalent to
$$
\dot{B}(xr,xo) =
  \begin{bmatrix}
    \frac{2}{a^2} (xr_0 - xo_0)\\
    \frac{2}{b^2} (xr_1 - xo_1)
  \end{bmatrix}^T
  \cdot \dot{xr} +
  \begin{bmatrix}
    \frac{2}{a^2} (xo_0 - xr_0)\\
    \frac{2}{b^2} (xo_1 - xr_1)
  \end{bmatrix}^T
  \cdot \dot{xo} \tag{3}
$$
(The expansion is shown in [02-DerivativeB.md](./02-DerivativeB.md))

here, $\dot{xr}$ is our control input and lets consider an example where the obstacle moves only left. That is $\dot{xo} = \begin{bmatrix} -1 \\ 0 \end{bmatrix}$.
$$
\dot{B}(xr,xo) =
  \begin{bmatrix}
    \frac{2}{a^2} (xr_0 - xo_0)\\
    \frac{2}{b^2} (xr_1 - xo_1)
  \end{bmatrix}^T
  \cdot
  \begin{bmatrix}
    u_0\\
    u_1
  \end{bmatrix}
  +
  \begin{bmatrix}
    \frac{2}{a^2} (xo_0 - xr_0)\\
    \frac{2}{b^2} (xo_1 - xr_1)
  \end{bmatrix}^T
  \cdot
  \begin{bmatrix}
    -1\\
    0
  \end{bmatrix}
$$
To make our implementation easier, we can also represent this in an augmented state
$$
x =
\begin{bmatrix}
  xr_0\\
  xr_1\\
  xo_0\\
  xo_1
\end{bmatrix}
$$
And in this case
$$
\dot{B}(x) =
  \begin{bmatrix}
    \frac{2}{a^2} (xr_0 - xo_0)\\
    \frac{2}{b^2} (xr_1 - xo_1)\\
    \frac{2}{a^2} (xo_0 - xr_0)\\
    \frac{2}{b^2} (xo_1 - xr_1)
  \end{bmatrix}^T
  \cdot
  \begin{bmatrix}
    u_0\\
    u_1\\
    -1\\
    0
  \end{bmatrix}
$$
Now, we solve the following quadratic program
$$
\min_{u} \quad ||u_{ref} - u||\\
\textrm{s.t.} \quad
\begin{bmatrix}
  \frac{2}{a^2} (xr_0 - xo_0)\\
  \frac{2}{b^2} (xr_1 - xo_1)\\
  \frac{2}{a^2} (xo_0 - xr_0)\\
  \frac{2}{b^2} (xo_1 - xr_1)
\end{bmatrix}^T
\cdot
\begin{bmatrix}
  u_0\\
  u_1\\
  -1\\
  0
\end{bmatrix}
\ge -(\frac{(xr_0-xo_0)^2}{a^2} + \frac{(xr_1-xo_1)^2}{b^2} - 1) \tag{4}
$$
We can pose this a quadratic program:
$$
\min_{\mathbf{x}} \quad  \frac{1}{2}\mathbf{x}^T P\mathbf{x} + q^T\mathbf{x} \\
\textrm{s.t.} \quad  G\mathbf{x} \le h \tag{5}
$$
In our case, $\mathbf{x} = \dot{x}$ is the decision variable,
Where
$$
P = I_4 \tag{6}
$$
$$
q = \begin{bmatrix}
      -u_{ref0}\\
      -u_{ref1}\\
      0\\
      0
    \end{bmatrix} \tag{7}
$$
$$
G = \begin{bmatrix}
      \frac{2}{a^2} (xr_0 - xo_0)\\
      \frac{2}{b^2} (xr_1 - xo_1)\\
      \frac{2}{a^2} (xo_0 - xr_0)\\
      \frac{2}{b^2} (xo_1 - xr_1)
    \end{bmatrix}^T \tag{8}
$$
$$
h = (\frac{(xr_0-xo_0)^2}{a^2} + \frac{(xr_1-xo_1)^2}{b^2} - 1) \tag{9}
$$
(The expansion is show in [02-CBF2Quadratic2.md](./02-CBF2Quadratic2.md))

Lets consider an example where our nimble ant system starts at position $xr = \begin{bmatrix} 0 \\  3.2 \end{bmatrix}$ and an obstacle starts at postion $xo = \begin{bmatrix} xo_0 \\  xo_1 \end{bmatrix} = \begin{bmatrix} 10 \\  3.5 \end{bmatrix}$.


## Code implementation

First, we import the necessary packages for our example.

In [ ]:
%matplotlib ipympl
from sympy import symbols, Matrix, sin, cos, lambdify, exp, sqrt, log, diff, Mul, srepr
from matplotlib.patches import Ellipse
import matplotlib.animation as animation
from cbfkit.tutorial import cbf as cbf, cbf_utils, sys_and_ctrl
import matplotlib.pyplot as plt
import numpy as np
from sympy import symbols

# import conditional if system is mac m1
if platform.system() == "Darwin" and platform.machine() == "arm64":
    from kvxopt import solvers
else:
    from cvxopt import solvers


In [ ]:
# Robot Goal
x_goal = np.array([10, 3.5])

# The bad set defines the radii of the eclipse. The position is part of the dynamics of the system.
bad_sets = [[3, 2]]

# Parameters for reference controller
ctrl_param = 0.3

# Symbols and equations for the CBF
xr_0, xr_1, xo_0, xo_1, a, b, xr_0_dot, xr_1_dot = symbols(
    'xr_0 xr_1 xo_0 xo_1 a b xr_0_dot xr_1_dot')
symbs = (xr_0, xr_1, xo_0, xo_1, a, b)

# Barrier function - distance of robot to obstacle
B = ((xr_0 - xo_0)/a)**2 + ((xr_1 - xo_1)/b)**2 - 1

# dx = f(x) + g(x)u
f = Matrix([0, 0, -1.0, 0])
g = Matrix([xr_0, xr_1, 0, 0])
degree = 1


In [ ]:
# Initialize CBF
my_CBF = cbf.CBF(B=B, f=f, g=g, states=(
    xr_0, xr_1, xo_0, xo_1), bad_sets=bad_sets, symbs=symbs, degree=1)

# Simulation settings
T_max = 20
n_samples = 100
T = np.linspace(0, T_max, n_samples)
dt = T[1]-T[0]
params = {'x_goal': x_goal, 'bad_sets': bad_sets,
          'ctrl_param': ctrl_param, 'CBF': my_CBF}


In [ ]:
# intial condition
x_0 = np.array([0, 3.2, 10, 3.5])

# Disable cvxopt optimiztaion output
solvers.options['show_progress'] = False
solvers.options['max_iter'] = 1000


In [ ]:
# Simulate system
print('\nComputing trajectories for the initial condition:')
print('x_0\t x_1')
print(x_0[0], '\t', x_0[1])

# Compute output on the nimble ant system for given initial conditions and timesteps T
x = np.zeros((np.size(x_0), len(T)))
x[:, 0] = x_0

# Simulate system
for i in range(len(T)-1):
    x[:, i+1] = x[:, i] + dt * \
        np.array(sys_and_ctrl.nimble_ant_with_agent_f(
            T[i], x[:, i], [], params))

print("\n*Simulation Done. See plot for animation...")


In [ ]:
# Animate
# Init Plot
fig, ax = plt.subplots(figsize=(6,6))
plt.xlim(0, 12)
plt.ylim(0, 6)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
line1, = ax.plot([], [], lw=2)
line2, = ax.plot([], [], lw=2)

ell = Ellipse((x[2][i-1], x[3][i-1]),
              bad_sets[0][0], bad_sets[0][1], color='g', alpha=0.3)

def animate(i):
    line1.set_data((x[0][0:i], x[1][0:i]))
    line2.set_data((x[2][0:i], x[3][0:i]))
    ell.center = (x[2][i], x[3][i])
    return line1, line2, ax.add_patch(ell)

plt.close()

ani = animation.FuncAnimation(
    fig, animate, interval=60, frames=n_samples, repeat=True, blit=True)

from IPython.display import HTML
from matplotlib import rcParams
rcParams['animation.html'] = 'html5'
HTML(ani.to_html5_video())
ani


To run the code directly in python see [02-Tutorial-CFB-Dynamic-Obstacles.py](./02-Tutorial-CFB-Dynamic-Obstacles.py) and [02A-Tutorial-CFB-Dynamic-Obstacles.py](./02A-Tutorial-CFB-Dynamic-Obstacles.py)